In [1]:
options(warn = -1)

In [2]:
library(lubridate)
library(rgdal)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [3]:
data <- read.csv('T:/Tableau/tableauBikesOnBuses/Datasources/MonthlyBikesOnBuses.csv')

In [4]:
names(data)

[1] "stop"      "srv"       "date"      "block"     "trip_end"  "time"     
 [7] "route"     "dir"       "stop_name" "bus"       "odometer"  "desc"     
[13] "qty"       "MonthYear" "longitude" "latitude"

In [5]:
head(data)

,stop,srv,date,block,trip_end,time,route,dir,stop_name,bus,odometer,desc,qty,MonthYear,longitude,latitude
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,<chr>,<dbl>,<dbl>
1,1,wkd,2013-01-18 00:00:00,67,1899-12-31 18:21:00,1899-12-31 18:10:00,11,O,E/S of 58th N of Main,1002,98.11,bike on rack,1,January 2013,-122.9267,44.04634
2,1,wkd,2013-01-10 00:00:00,2,1899-12-31 09:56:00,1899-12-31 09:44:00,11,O,E/S of 58th N of Main,1114,71.08,bike on rack,1,January 2013,-122.9267,44.04634
3,1,wkd,2013-01-10 00:00:00,60,1899-12-31 16:54:00,1899-12-31 16:45:00,11,O,E/S of 58th N of Main,7102,119.29,bike on rack,1,January 2013,-122.9267,44.04634
4,1,wkd,2013-01-29 00:00:00,60,1899-12-31 17:52:00,1899-12-31 17:39:00,11,O,E/S of 58th N of Main,7102,132.76,bike on rack,1,January 2013,-122.9267,44.04634
5,1,wkd,2013-01-09 00:00:00,25,1899-12-31 15:35:00,1899-12-31 15:20:00,11,O,E/S of 58th N of Main,1004,126.65,bike on rack,1,January 2013,-122.9267,44.04634
6,2,wkd,2013-01-30 00:00:00,48,1899-12-31 08:27:00,1899-12-31 08:13:00,11,O,E/S of 58th S of D,6218,5.11,bike on rack,1,January 2013,-122.9262,44.04946


In [6]:
data$date <- as.Date(data$date, "%Y-%m-%d")

In [7]:
data$year <- year(data$date)

In [8]:
dat <- data[data$year == 2021,]

In [9]:
dim(dat)

[1] 49945    17

In [10]:
dat$weekday <- as.character(wday(dat$date, label=TRUE, abbr=FALSE))

In [11]:
dat$month <- months(dat$date)

In [12]:
dat$season <- ifelse(dat$month %in% c("December", "January", "February"), "Winter",
                    ifelse(dat$month %in% c("March", "April", "May"), "Spring",
                          ifelse(dat$month %in% c("June", "July", "August"), "Summer", "Fall")))

In [13]:
dat$seasonOrder <- ifelse(dat$season == "Spring", 1, 
                              ifelse(dat$season == "Summer", 2, 
                                     ifelse(dat$season == "Fall", 3, 4)))

In [14]:
dat$monthOrder <- ifelse(dat$month == "January", 1, 
                           ifelse(dat$month == "February", 2, 
                               ifelse(dat$month == "March", 3, 
                                   ifelse(dat$month == "April", 4, 
                                       ifelse(dat$month == "May", 5, 
                                           ifelse(dat$month == "June", 6, 
                                               ifelse(dat$month == "July", 7, 
                                                   ifelse(dat$month == "August", 8, 
                                                       ifelse(dat$month == "September", 9, 
                                                           ifelse(dat$month == "October", 10, 
                                                               ifelse(dat$month == "November", 11, 12)))))))))))

In [15]:
dat$weekdayOrder <- ifelse(dat$weekday == "Monday", 1, 
                              ifelse(dat$weekday == "Tuesday", 2, 
                                     ifelse(dat$weekday == "Wednesday", 3, 
                                     ifelse(dat$weekday == "Thursday", 4, 
                                            ifelse(dat$weekday == "Friday", 5, 
                                                ifelse(dat$weekday == "Saturday", 6, 7))))))

In [16]:
datedf <- unique(dat[,c('date', 'season', 'month', 'weekday', 'seasonOrder', 'monthOrder', 'weekdayOrder')])

In [17]:
stop.path <- "T:/Data/LTD Data/StopsSince2011"

In [18]:
stops <- readOGR(dsn = stop.path, layer = "April 2020", verbose = FALSE, 
                   stringsAsFactors = FALSE)

In [19]:
names(stops)

[1] "stop_numbe" "stop_name"  "latitude"   "longitude"  "AvgBoards" 
[6] "city"

In [20]:
unique(stops$city)

[1] "Springfield" "Eugene"      NA            "Coburg"

In [21]:
dim(stops@data)[1] == length(unique(stops$stop_numbe))

[1] TRUE

In [22]:
head(data$stop)

[1] 1 1 1 1 1 2

In [23]:
head(stops$stop_numbe)

[1] "00001" "00002" "00004" "00006" "00007" "00008"

In [24]:
zeros <- c("0", "00", "000", "0000")

In [25]:
dat$stop <- ifelse(nchar(dat$stop) == 5, dat$stop,
                    paste0(zeros[(5 - nchar(dat$stop))], dat$stop))

In [26]:
head(dat)

,stop,srv,date,block,trip_end,time,route,dir,stop_name,bus,⋯,MonthYear,longitude,latitude,year,weekday,month,season,seasonOrder,monthOrder,weekdayOrder
,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
782796,00001,sat,2021-01-23,7029,1899-12-31 14:35:00,1899-12-31 14:22:00,11,O,E/S of 58th N of Main,1122,⋯,January 2021,-122.9267,44.04634,2021,Saturday,January,Winter,4,1,6
782797,00001,wkd,2021-01-21,52,1899-12-31 17:07:00,1899-12-31 16:56:00,11,O,E/S of 58th N of Main,19106,⋯,January 2021,-122.9267,44.04634,2021,Thursday,January,Winter,4,1,4
782798,00001,wkd,2021-01-29,70,1899-12-31 18:34:00,1899-12-31 18:21:00,11,O,E/S of 58th N of Main,1402,⋯,January 2021,-122.9267,44.04634,2021,Friday,January,Winter,4,1,5
782799,00001,wkd,2021-01-05,32,1899-12-31 23:04:00,1899-12-31 22:56:00,11,O,E/S of 58th N of Main,19102,⋯,January 2021,-122.9267,44.04634,2021,Tuesday,January,Winter,4,1,2
782800,00001,wkd,2021-01-12,32,1899-12-31 16:53:00,1899-12-31 16:44:00,11,O,E/S of 58th N of Main,1001,⋯,January 2021,-122.9267,44.04634,2021,Tuesday,January,Winter,4,1,2
782801,00004,wkd,2021-01-13,29,1899-12-31 13:38:00,1899-12-31 13:26:00,11,O,E/S of 58th S of Thurston Rd,1102,⋯,January 2021,-122.9262,44.05225,2021,Wednesday,January,Winter,4,1,3


In [27]:
names(stops@data)

[1] "stop_numbe" "stop_name"  "latitude"   "longitude"  "AvgBoards" 
[6] "city"

In [28]:
stopdf <- unique(dat[,c('stop', 'route')])

In [29]:
length(unique(stopdf$stop))

[1] 842

In [30]:
names(stops)[1] <- "stop"

In [31]:
dim(stops@data)

[1] 1245    6

In [32]:
length(unique(stops@data$stop))

[1] 1245

In [33]:
stopdf <- merge(stops@data[, -which(names(stops@data)=="AvgBoards")], stopdf, by="stop")

In [34]:
dim(stopdf)

[1] 983   6

In [35]:
length(unique(stopdf$stop))

[1] 842

In [36]:
dim(unique(stopdf[,c("stop", "route")]))

[1] 983   2

In [37]:
head(stopdf)

,stop,stop_name,latitude,longitude,city,route
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,00001,E/S of 58th N of Main,44.04634,-122.9267,Springfield,11
2,00002,E/S of 58th S of D,44.04946,-122.9262,Springfield,11
3,00004,E/S of 58th S of Thurston Rd,44.05225,-122.9262,Springfield,11
4,00006,S/S of Thurston W of 64th,44.05295,-122.9166,Springfield,11
5,00007,S/S of Thurston E of 65th,44.05267,-122.9132,Springfield,11
6,00008,S/S of Thurston W of 66th,44.05324,-122.9113,Springfield,11


In [38]:
unique(stopdf$city)

[1] "Springfield" "Eugene"      NA            "Coburg"

In [39]:
stopdf <- stopdf[!is.na((stopdf$city)),]

In [40]:
aggdata <- aggregate(x=list(qty = dat$qty), 
                     by=list(date = dat$date, stop = dat$stop, route=dat$route), 
                     FUN=sum)

In [41]:
dim(aggdata)

[1] 35222     4

In [42]:
head(aggdata)

,date,stop,route,qty
,<date>,<chr>,<chr>,<int>
1,2021-08-02,01058,1,1
2,2021-08-13,01058,1,1
3,2021-10-06,01058,1,1
4,2021-10-15,01058,1,1
5,2021-01-23,01060,1,1
6,2021-05-24,01060,1,1


In [43]:
get_aggdata <- function(df=dat, dir="O"){
    df <- df[df$dir == dir,]
    aggdata <- aggregate(x=list(qty = df$qty), 
                     by=list(date = df$date, stop = df$stop, route=df$route), 
                     FUN=sum)

    if(dir=="O"){
        names(aggdata)[4] <- "out_qty"
    }else{
        names(aggdata)[4] <- "in_qty"
    }
    
    return(aggdata)    
}

In [44]:
out_df <- get_aggdata(df=dat, dir="O")

In [45]:
dim(out_df)

[1] 17375     4

In [46]:
in_df <- get_aggdata(df=dat, dir="I")

In [47]:
dim(in_df)

[1] 17865     4

In [48]:
aggdata <- merge(aggdata, get_aggdata(df=dat, dir="O"), by=c("date", "stop", "route"), all=TRUE)

In [49]:
dim(aggdata )

[1] 35222     5

In [50]:
aggdata <- merge(aggdata, get_aggdata(df=dat, dir="I"), by=c("date","stop", "route"), all=TRUE)

In [51]:
aggdata <- merge(aggdata, stopdf, by=c("stop", "route"))

In [52]:
aggdata <- merge(aggdata, datedf, by="date")

In [53]:
outpath <- "T:/DCProjects/StoryMap/BikeCounting/BikeOnBuses/Output"

In [54]:
write.csv(aggdata, paste0(outpath, "/Daily_Bikes_On_Buses.csv"), row.names = FALSE)

In [55]:
MPOBound <- readOGR(dsn = "V:/Data/Transportation", layer="MPO_Bound")

OGR data source with driver: ESRI Shapefile 
Source: "V:\Data\Transportation", layer: "MPO_Bound"
with 1 features
It has 3 fields


In [56]:
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf, CRS(proj4string(MPOBound)))
  }
  return(spdf)
}

In [57]:
spdf <- df2spdf(aggdata, 'longitude', 'latitude')

In [58]:
writeOGR(spdf, dsn=outpath, layer="Daily_Bikes_On_Buses", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [59]:
sumdf <- aggregate(x=list(qty = aggdata$qty, 
                          out_qty = aggdata$out_qty, 
                          in_qty = aggdata$in_qty), 
                   by=list(stop = aggdata$stop, route=aggdata$route), 
                   FUN=sum)

In [60]:
avgdf <- aggregate(x=list(avgqty = aggdata$qty), 
                   by=list(stop = aggdata$stop, route=aggdata$route), 
                   FUN=mean)

In [61]:
sumdf$pct_out <- (sumdf$out_qty / sumdf$qty) * 100

In [62]:
sum_avg_df <- merge(sumdf, avgdf, by=c("stop", "route"))

In [63]:
sum_avg_df <- merge(sum_avg_df, stopdf, by=c("stop", "route"))

In [84]:
range(sum_avg_df$avgqty)

[1] 1.000000 7.388268

In [64]:
dim(sum_avg_df)

[1] 898  11

In [69]:
names(sum_avg_df)

[1] "stop"      "route"     "qty"       "out_qty"   "in_qty"    "pct_out"  
 [7] "avgqty"    "stop_name" "latitude"  "longitude" "city"

In [70]:
route_avg <- aggregate(x=list(avgqty=sum_avg_df$avgqty), by=list(route=sum_avg_df$route), FUN=mean)

In [85]:
range(route_avg$avgqty)

[1] 1.000000 1.443346

In [79]:
winter20routes <- readOGR(dsn="T:/Data/LTD Data/2020 Winter LTD Routes and Stops", layer="Winter_2020_Routes")

OGR data source with driver: ESRI Shapefile 
Source: "T:\Data\LTD Data\2020 Winter LTD Routes and Stops", layer: "Winter_2020_Routes"
with 31 features
It has 2 fields


In [80]:
head(winter20routes@data)

,ROUTE,Shape_Leng
,<chr>,<dbl>
0,1,7022.793
1,11,49572.562
2,12,43357.313
3,13,40336.364
4,17,23944.353
5,18,21478.748


In [81]:
names(winter20routes@data)[1] <- "route"

In [82]:
winter20routes <- merge(winter20routes, route_avg, by="route")

In [65]:
write.csv(sum_avg_df, paste0(outpath, "/Sum_Bikes_On_Buses.csv"), row.names = FALSE)

In [83]:
writeOGR(winter20routes, dsn=outpath, layer="Winter_2020_Routes", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [66]:
sum_avg_spdf <- df2spdf(sum_avg_df, 'longitude', 'latitude')

In [67]:
names(sum_avg_spdf)

[1] "stop"      "route"     "qty"       "out_qty"   "in_qty"    "pct_out"  
 [7] "avgqty"    "stop_name" "latitude"  "longitude" "city"

In [68]:
writeOGR(sum_avg_spdf, dsn=outpath, layer="Sum_Bikes_On_Buses", driver="ESRI Shapefile", overwrite_layer=TRUE)